### We'd be looking at the topics below in this notebook:

1. Different models
2. Structured outputs
3. Guardrails

In [29]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, output_guardrail
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [3]:
load_dotenv(override=True)

True

In [4]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


In [5]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [7]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [13]:
print(deepseek_client)
print(deepseek_model)

In [14]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [15]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [16]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = os.getenv("YAHOO_ID")
    to_email = os.getenv("GMAIL_ID")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [17]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [18]:
email_tools = [subject_tool, html_tool, send_html_email]

In [19]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [20]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [21]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

#### Implementing Guardrails

In [22]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [23]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [24]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

### Implementing Output guardrail for practice

In [ ]:
# class no_company_name_in_message(BaseModel):
#     is_company_name_in_message: bool
#     company_name: str


# guardrail_agent_2 = Agent( 
#     name="Company Name check",
#     instructions="Check if the agent is including the company's name in email generated by it",
#     output_type=no_company_name_in_message,
#     model="gpt-4o-mini"
# )


# @output_guardrail
# async def guardrail_against_companyname(ctx, agent, result):
#     # generated = result.final_output
#     # result = await Runner.run(guardrail_agent_2, result, context=ctx.context)
#     # companyname_in_message = result.final_output.is_company_name_in_message
#     # return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=companyname_in_message)
#      # Extract the text the agent produced. Adjust this if your agent returns a structured object.
#     generated = getattr(result, "final_output", result)

#     # If the agent returned a dict/object, pick the right field that contains the email text
#     if isinstance(generated, dict) and "text" in generated:
#         generated = generated["text"]

#     # Defensive: ensure we pass a string to the checker
#     generated_str = str(generated)

#     check = await Runner.run(guardrail_agent_2, generated_str, context=ctx.context)
#     trip = bool(check.final_output.is_company_name_in_message)

#     return GuardrailFunctionOutput(
#         output_info={"company_check": check.final_output},
#         tripwire_triggered=trip
#     )


# careful_sales_manager = Agent(
#     name="Sales Manager",
#     instructions=sales_manager_instructions,
#     tools=tools,
#     handoffs=[emailer_agent],
#     model="gpt-4o-mini",
#     # input_guardrails=[guardrail_against_name]
#     output_guardrails=[guardrail_against_companyname]
#     )

# message = "Send out a cold sales email addressed to Dear CEO from Alice"

# with trace("Protected Automated SDR"):
#     result = await Runner.run(careful_sales_manager, message)

In [35]:
# from pydantic import BaseModel, Field

# # 1) Schema for the guardrail agent's structured output
# class CompanyNameCheck(BaseModel):
#     is_company_name_in_message: bool = Field(..., description="True if a company name is present")
#     company_name: str = Field("", description="Detected company name, else empty")

# # 2) Guardrail agent that inspects the *generated email text*
# guardrail_agent_2 = Agent(
#     name="Company Name check",
#     instructions=(
#         "You are a strict checker. Given an email body, respond in JSON: "
#         "{ is_company_name_in_message: bool, company_name: string }. "
#         "Return company_name empty when not present."
#     ),
#     output_type=CompanyNameCheck,
#     model="gpt-4o-mini",
# )

# # 3) OUTPUT guardrail runs AFTER the agent produces a result
# @output_guardrail
# async def guardrail_against_companyname(ctx, agent, result):
#     """
#     NOTE: For output guardrails, you receive the agent's *result* (not the input message).
#     `result` is typically a RunResult with `final_output`.
#     """
#     # Extract the text the agent produced. Adjust this if your agent returns a structured object.
#     generated = getattr(result, "final_output", result)

#     # If the agent returned a dict/object, pick the right field that contains the email text
#     if isinstance(generated, dict) and "text" in generated:
#         generated = generated["text"]

#     # Defensive: ensure we pass a string to the checker
#     generated_str = str(generated)

#     check = await Runner.run(guardrail_agent_2, generated_str, context=ctx.context)
#     trip = bool(check.final_output.is_company_name_in_message)

#     return GuardrailFunctionOutput(
#         output_info={"company_check": check.final_output},
#         tripwire_triggered=trip
#     )


from pydantic import BaseModel, Field

class ToneCheckOutput(BaseModel):
    is_tone_acceptable: bool = Field(..., description="False if the tone is too pushy, spammy, or inappropriate")
    reason: str = Field("", description="Why the tone was flagged")

tone_guardrail_agent = Agent(
    name="Tone Check",
    instructions=(
        "You are a tone auditor. Given a sales email, decide if it follows these rules:\n"
        "- Polite and professional.\n"
        "- No exaggerated claims or spammy language.\n"
        "- Avoid ALL CAPS or multiple exclamation points.\n\n"
        "Return JSON: { is_tone_acceptable: bool, reason: str }"
    ),
    output_type=ToneCheckOutput,
    model="gpt-4o-mini"
)

@output_guardrail
async def guardrail_against_bad_tone(ctx, agent, result):
    generated = getattr(result, "final_output", result)
    email_text = str(generated.get("body") if isinstance(generated, dict) else generated)

    check = await Runner.run(tone_guardrail_agent, email_text, context=ctx.context)
    trip = not check.final_output.is_tone_acceptable

    return GuardrailFunctionOutput(
        output_info={"tone": check.final_output},
        tripwire_triggered=trip
    )






# We will create another email agent so that we can attach the output guardrail to it.
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it",
    output_guardrails=[guardrail_against_bad_tone] )


# Attach to the agent that produces the DRAFT to be sent.
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
)

message = "Send out a cold sales email addressed to Dear CEO from Alice with bad tone and spammy language!!!"
with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)


OutputGuardrailTripwireTriggered: Guardrail OutputGuardrail triggered tripwire